<a href="https://colab.research.google.com/github/ag497/Heston-Stochastic-Volatility-Model/blob/main/ASHISH_HESTON_STOCHASTIC_VOLATILITY_MODEL_CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import scipy
from scipy.stats import norm
from scipy.integrate import quad

In [ ]:
crm = yf.Ticker("CRM")
hist = crm.history(period="max")
hist.drop(["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"], axis = 1, inplace = True )
hist.rename(columns = {"Close":"S"}, inplace = True)
hist

,S
Date,
2004-06-23 00:00:00-04:00,4.294391
2004-06-24 00:00:00-04:00,4.184534
2004-06-25 00:00:00-04:00,3.944847
2004-06-28 00:00:00-04:00,3.994782
2004-06-29 00:00:00-04:00,4.094651
...,...
2024-06-14 00:00:00-04:00,231.940002
2024-06-17 00:00:00-04:00,230.479996
2024-06-18 00:00:00-04:00,231.809998


In [ ]:
hist["Q"] = hist["S"]/hist["S"].shift(1)
hist.dropna(axis = 0, inplace = True)
hist

,S,Q
Date,,
2004-06-24 00:00:00-04:00,4.184534,0.974419
2004-06-25 00:00:00-04:00,3.944847,0.942721
2004-06-28 00:00:00-04:00,3.994782,1.012658
2004-06-29 00:00:00-04:00,4.094651,1.025000
2004-06-30 00:00:00-04:00,4.012259,0.979878
...,...,...
2024-06-14 00:00:00-04:00,231.940002,1.012706
2024-06-17 00:00:00-04:00,230.479996,0.993705
2024-06-18 00:00:00-04:00,231.809998,1.005771


In [ ]:
arr = np.array(hist['Q'])
from scipy import stats
mews = []
for i in range(0, 5):
  mews.append(np.mean(arr**(i+1)))
mews



[1.0011559885027115,
 1.0030195606935866,
 1.0055997015858003,
 1.008911953992891,
 1.0129785363578887]

In [ ]:
from scipy.optimize import minimize

# Given equations as functions of k, r, sigma, theta
def equations(vars):
    k, r, sigma, theta = vars
    mu1 = 1 + r
    mu2 = (r + 1)**2 + theta
    mu4 = (1 / (k * (k - 2))) * (
        k**2 * r**4 + 4 * k**2 * r**3 + 6 * k**2 * r**2 * theta - 2 * k * r**4 + 6 * k**2 * r**2 + 12 * k**2 * r * theta
        + 3 * k**2 * theta**2 - 8 * k * r**3 - 12 * k * r * theta + 4 * k**2 * r + 6 * k**2 * theta - 12 * k * r**2
        - 24 * k * r * theta - 6 * k * theta**2 - 3 * sigma**2 * theta + k**2 - 8 * k * r - 12 * k * theta - 2 * k
    )
    mu5 = (1 / (k * (k - 2))) * (
        k**2 * r**5 + 5 * k**2 * r**4 + 10 * k**2 * r**3 * theta - 2 * k * r**5 + 10 * k**2 * r**3 + 30 * k**2 * r**2 * theta
        + 15 * k**2 * r * theta**2 - 10 * k * r**4 - 20 * k * r**3 * theta + 10 * k**2 * r**2 + 30 * k**2 * r * theta + 15 * k**2 * theta**2
        - 20 * k * r**3 - 60 * k * r**2 * theta - 30 * k * r * theta**2 - 15 * sigma**2 * theta + 5 * k**2 * r + 10 * k**2 * theta
        - 20 * k * r**2 - 60 * k * r * theta - 30 * k * theta**2 - 15 * sigma**2 * theta + k**2 - 10 * k * r - 20 * k * theta - 2 * k
    )
    return mu1, mu2, mu4, mu5

# Objective function for optimization
def objective(vars):
    k, r, sigma, theta = vars
    mu1, mu2, mu4, mu5 = equations(vars)
    target_mu1 = mews[0]
    target_mu2 = mews[1]
    target_mu4 = mews[3]
    target_mu5 = mews[4]
    return (mu1 - target_mu1)**2 + (mu2 - target_mu2)**2 + (mu4 - target_mu4)**2 + (mu5 - target_mu5)**2

# Constraints
constraints = (
    {'type': 'ineq', 'fun': lambda vars: vars[2]**2},  # sigma^2 > 0
    {'type': 'ineq', 'fun': lambda vars: 2 * vars[0] * vars[3] - vars[2]**2}  # 2k*theta > sigma^2
)

# Initial guess
initial_guess = [1, 1, 1, 1]

# Solve the optimization problem
result = minimize(objective, initial_guess, constraints=constraints)

if result.success:
    k_opt, r_opt, sigma_opt, theta_opt = result.x
    print(f'Optimal values: k = {k_opt:.6f}, r = {r_opt:.6f}, sigma = {sigma_opt:.6f}, theta = {theta_opt:.6f}')
else:
    print('Optimization failed.')



Optimal values: k = 1.750263, r = 0.000843, sigma = 0.006110, theta = 0.000929


In [ ]:
import numpy as np
from math import sqrt

i = complex(0, 1)

def c_fun(u, S_t, K, r, tau, kappa, theta, sigma, rho, v_t):
    M = np.sqrt((rho * sigma * 1j * u - kappa)**2 + sigma**2 * (u * 1j + u**2))
    N = (rho * sigma * 1j * u - kappa - M) / (rho * sigma * 1j * u - kappa + M)
    A = r * 1j * u * tau + (kappa * theta) / sigma**2 * (-(rho * sigma * 1j * u - kappa - M) * tau - 2 * np.log((1 - N * np.exp(-M * tau)) / (1 - N)))
    C = ((rho * sigma * 1j * u - kappa - M) * (np.exp(M*tau) - 1)) / (sigma**2 * (1 - N * np.exp(M * tau)))
    heston_cf = np.exp(A + C * v_t + 1j * u * np.log(S_t))

    return heston_cf

def heston_call_price(St, K, r, tau, kappa, theta, sigma, rho, vt):
    P1 = 0
    iterations = 1000
    maxNumber = 100
    ds = maxNumber / iterations

    for j in range(1, iterations):
        s1 = ds * (2 * j + 1) / 2
        s2 = s1 - i
        numerator1 = c_fun(s2, St, K, r, tau, kappa, theta, sigma, rho, vt)
        numerator2 = K * c_fun(s1, St, K, r, tau, kappa, theta, sigma, rho, vt)
        denominator = np.exp(np.log(K) * i * s1) * i * s1

        P1 += ds * (numerator1 - numerator2) / denominator

    P1 = np.real(P1 / np.pi)
    P2 = P1

    call_price = St * (0.5 + P1) - K * np.exp(-r * tau) * (0.5 + P2)

    return call_price


In [ ]:
# Calculate logarithmic returns
hist['log_ret'] = np.log(hist['S'] / hist['S'].shift(1))
# Calculate volatility as the difference between latest log return and mean

mean = hist['log_ret'].mean()       # Mean log return
latest = hist['log_ret'].iloc[-1]  # Latest log return
volatility = latest - mean
S_t = hist.iloc[-1]['S']
K = 245
r = ( ( 1 + r_opt)**252) - 1 # Adjusted for annualization
tau = 5/365
kappa = k_opt
theta = theta_opt
sigma = sigma_opt * sqrt(252)  # Adjusted for annualization
rho = 0.8
v_t = volatility

call_price = heston_call_price(S_t, K, r, tau, kappa, theta, sigma, rho, v_t)
print(f"Heston Call Price According To Method Of Moments: ${call_price}")


Heston Call Price According To Method Of Moments: $1.7080293601143808


In [ ]:
import numpy as np
import pandas as pd


# hist['S'] contains the time series of stock prices

# Calculate log returns
hist['log_ret'] = np.log(hist['S'] / hist['S'].shift(1))

# Define the window for variance calculation
window = 252  # 252 trading days for annualized volatility

# Calculate Vt (variance of log returns)
hist['Vt'] = hist['log_ret'].rolling(window=window).var()

# Drop rows with NaN values in case there are any
hist.dropna(inplace=True)

# Display or use Vt for further analysis
print(hist[['S', 'log_ret', 'Vt']])

                                    S   log_ret        Vt
Date                                                     
2005-06-23 00:00:00-04:00    4.966013 -0.003012  0.001505
2005-06-24 00:00:00-04:00    5.193216  0.044736  0.001499
2005-06-27 00:00:00-04:00    5.053399 -0.027292  0.001501
2005-06-28 00:00:00-04:00    5.158262  0.020539  0.001501
2005-06-29 00:00:00-04:00    5.200706  0.008195  0.001499
...                               ...       ...       ...
2024-06-14 00:00:00-04:00  231.940002  0.012626  0.000484
2024-06-17 00:00:00-04:00  230.479996 -0.006315  0.000484
2024-06-18 00:00:00-04:00  231.809998  0.005754  0.000484
2024-06-20 00:00:00-04:00  241.800003  0.042193  0.000488
2024-06-21 00:00:00-04:00  245.059998  0.013392  0.000484

[4781 rows x 3 columns]


In [ ]:
import numpy as np
from scipy.optimize import minimize



# Likelihood function for Qt+1 and Vt+1
def log_likelihood(vars, Qt, Vt):
    r, k, theta, sigma, rho = vars
    n = len(Qt)
    log_likelihood_value = 0

    for t in range(n):
        if t == 0:
            Qt_prev = 1  # Assuming initial value for Qt is 1
            Vt_prev = np.var(Qt)  # Initial guess for Vt based on Qt
        else:
            Qt_prev = Qt[t - 1]
            Vt_prev = Vt[t - 1]

        Qt_curr = Qt[t]
        Vt_curr = Vt[t]

        # Joint probability density function
        # Check and adjust for positive values in logarithm arguments
        epsilon = 1e-8  # small epsilon to prevent zero values
        # Calculate part1, skipping undefined logarithms
        if sigma > 0 and Vt_prev > 0 and 1 - rho**2 > epsilon:
          part1 = -0.5 * np.log(2 * np.pi) - np.log(sigma) - 0.5 * np.log(Vt_prev) - 0.5 * np.log(1 - rho**2)
        else:
          part1 = 0  # or any other appropriate value or action when logarithm is undefined
        part2 = -0.5 * ((Qt_curr - (1 + r))**2) / (Vt_curr * (1 - rho**2))
        part3 = (rho * (Qt_curr - (1 + r)) * (Vt_curr - Vt_prev - k * (theta - Vt_prev))) / (Vt_curr * sigma * (1 - rho**2))
        part4 = -0.5 * ((Vt_curr - Vt_prev - k * (theta - Vt_prev))**2) / (sigma**2 * Vt_curr * (1 - rho**2))

        log_likelihood_value += part1 + part2 + part3 + part4

    return -log_likelihood_value  # Minimize negative log-likelihood

# Initial guess for parameters
initial_guess = [r_opt, k_opt, theta_opt, sigma_opt, 0.8]  # Initial guesses for r, k, theta, sigma, rho

# Solve the optimization problem
result = minimize(log_likelihood, initial_guess, args=(hist['Q'], hist['Vt']), method='L-BFGS-B')

if result.success:
    r_opt, k_opt, theta_opt, sigma_opt, rho_opt = result.x
    print(f'Optimal values: r = {r_opt:.6f}, k = {k_opt:.6f}, theta = {theta_opt:.6f}, sigma = {sigma_opt:.6f}, rho = {rho_opt:.6f}')
else:
    print('Optimization failed.')



Optimal values: r = 0.001029, k = 0.001042, theta = 0.000329, sigma = 0.000564, rho = 0.000594


In [ ]:
# Calculate logarithmic returns
hist['log_ret'] = np.log(hist['S'] / hist['S'].shift(1))
# Calculate volatility as the difference between latest log return and mean

mean = hist['log_ret'].mean()       # Mean log return
latest = hist['log_ret'].iloc[1]  # Latest log return
volatility = latest - mean

S_t = hist.iloc[-1]['S']
K = 245
r = ( ( 1 + r_opt)**252) - 1 # Adjusted for annualization
tau = 5/365
kappa = k_opt
theta = theta_opt
sigma = sigma_opt * sqrt(252)  # Adjusted for annualization
rho = rho_opt
v_t = volatility

call_price = heston_call_price(S_t, K, r, tau, kappa, theta, sigma, rho, v_t)
print(f"Heston Call Price According To MLE: ${call_price}")

Heston Call Price According To MLE: $3.0508224268404547
